In [1]:
import torchio as tio
import os
import nibabel as nib

In [4]:
source_mask_dir = "/work/disa_lab/projects/tiny_brains/unet_segmentation/unet_segmentation/SS"
source_data_dir = "/work/disa_lab/projects/tiny_brains/unet_segmentation/unet_segmentation/Images"

In [5]:
len(os.listdir(source_data_dir)), len(os.listdir(source_mask_dir))

(359, 359)

In [6]:
skull_stripped_destination_data_dir = "/work/disa_lab/projects/tiny_brains/unet_segmentation/unet_segmentation/skull_stripped_source_images"

In [7]:
destination_data_dir = "/work/disa_lab/projects/tiny_brains/unet_segmentation/unet_segmentation/cc_motion_corrupted"

In [8]:
os.makedirs(skull_stripped_destination_data_dir, exist_ok=True)
os.makedirs(destination_data_dir, exist_ok=True)

In [9]:
motion_transformation = tio.transforms.RandomMotion(
    degrees=5,
    translation=10,
    num_transforms=3
)

In [10]:
output_shape = (256, 288, 288)

crop_or_pad = tio.CropOrPad(output_shape)

In [11]:
source_images = os.listdir(source_data_dir)

In [12]:
def apply_mask(image, mask):
    image_array = image.get_fdata()
    mask_array = mask.get_fdata()

    mask_array = mask_array.astype(bool)

    masked_image_array = image_array * mask_array

    masked_image = nib.Nifti1Image(masked_image_array, image.affine)

    return masked_image

In [13]:
import time
from tqdm import tqdm

for image in tqdm(source_images, total=len(source_images)):
    print(f"Now processing: {image}")

    if image == ".DS_Store":
        continue
    
    start = time.time()

    three_d_image = nib.load(os.path.join(source_data_dir, image))
    image_mask = nib.load(os.path.join(source_mask_dir, image.split(".")[0] + "_staple" + ".nii"))

    three_d_image = apply_mask(three_d_image, image_mask)

    three_d_image = crop_or_pad(three_d_image)

    transformed_image = motion_transformation(three_d_image)
    nib.save(three_d_image, os.path.join(skull_stripped_destination_data_dir, image))
    nib.save(transformed_image, os.path.join(destination_data_dir, image))

    print(f"Time taken: {time.time() - start}")

  0%|          | 0/359 [00:00<?, ?it/s]

Now processing: CC0001_philips_15_55_M.nii


  0%|          | 1/359 [00:08<48:06,  8.06s/it]

Time taken: 8.06164836883545
Now processing: CC0002_philips_15_56_M.nii


  1%|          | 2/359 [00:15<45:28,  7.64s/it]

...

Time taken: 7.57104754447937
Now processing: CC0358_ge_3_52_M.nii


100%|█████████▉| 358/359 [44:44<00:07,  7.55s/it]

Time taken: 7.365015506744385
Now processing: CC0359_ge_3_58_F.nii


100%|██████████| 359/359 [44:51<00:00,  7.50s/it]

Time taken: 7.140428066253662


In [22]:
nib.load(destination_data_dir + "/" +os.listdir(skull_stripped_destination_data_dir)[120]).get_fdata().shape

(256, 288, 288)